In [37]:
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import gc
gc.collect()

62337

In [38]:
X = np.array([[2, -1], [3, -2], [1, 0], [0,1],[-2,1],[-1.3,0.3],[-0.2,-0.8],[2.3,-3.3],[-2,-4],[7,8]])
y = np.array([1, 1, 1, 1,-1,-1,-1,-1,-1,1])

In [39]:
#通过SMO算法进行优化求解
class SVC:
    def __init__(self, C=1, maxIter=10):
        self.C = C
        self.maxIter = maxIter
        self.K = None
        self.alpha = np.zeros(n_samples) #初始化alpha
        self.b = 0 #初始化b
        self.X = None
        self.y = None
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        n_samples = X.shape[0]
        alpha = self.alpha
        iter_num = 0
        self.K = computeK(X) #计算kernel
        K = self.K
        b = self.b
        
        while(iter_num < self.maxIter):
            u = self.compute_u(X, y)
            iter_num += 1
            finish = True
            for i in range(n_samples):
                if not self.checkKKT(u,y,i):  #若不满足KKT条件，则需要进行参数更新
                    finish = False
                    alpha_old = alpha.copy()   #旧值后续要用
                    y_indices = np.delete(np.arange(n_samples), i) #去除i
                    j = y_indices[int(np.random.random()*len(y_indices))] #随机选择除i外的j

                    #计算误差
                    Ei = np.sum(alpha*y*K[:, i])+b-y[i] #对应元素相乘，所以用*
                    Ej = np.sum(alpha*y*K[:, j])+b-y[j]

                    #计算上下边界
                    if y[i] != y[j]:
                        L = max(0, alpha[j] - alpha[i])
                        H = min(self.C, self.C + alpha[j] - alpha[i])
                    else:
                        L = max(0, alpha[j] + alpha[i] - self.C)
                        H = min(self.C, alpha[j] + alpha[i])

                    #计算eta
                    eta = K[i, i] + K[j, j] - 2*K[i, j]

                    #更新alpha j
                    alpha[j] = alpha[j] + y[j]*(Ei-Ej) / eta

                    #根据取值范围修剪alpha j
                    if alpha[j] > H:
                        alpha[j] = H
                    elif (alpha[j] >=L) and (alpha[j] <=H):
                        pass
                    else:
                        alpha[j] = L

                    #更新alpha i
                    alpha[i] = alpha[i] + y[i]*y[j]*(alpha[j] - alpha_old[j])

                    #更新b
                    bi = -Ei - y[i]*K[i, i]*(alpha[i] - alpha_old[i]) - y[j]*K[j, i]*(alpha[j] - alpha_old[j]) + b
                    bj = -Ej - y[i]*K[i, j]*(alpha[i] - alpha_old[i]) - y[j]*K[j, i]*(alpha[j] - alpha_old[j]) + b

                    if (alpha[i]>0) and (alpha[i]<self.C):
                        b = bi
                    elif (alpha[j]>0) and (alpha[j]<self.C):
                        b = bj
                    else:
                        b = np.mean(bi+bj)
                    
                    self.alpha = alpha
                    self.b = b
            if finish:
                break
                
    def predict(self,X):
        y_preds=[]
        for i in range(X.shape[0]):
            K=np.zeros((len(self.y),))
            support_indices=np.where(self.alpha>0)[0]
            for j in support_indices:
                K[j]=self.X[j].dot(X[i])
            y_pred=np.sum(self.y[support_indices]*self.alpha[support_indices]*K[support_indices].T)
            y_pred+=self.b
            y_preds.append(y_pred)
        return np.array(y_preds)

        

    def computeK(self, X):  #计算x的所有相关k
        n_samples = X.shape[0]
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                if i<=j:
                    K[i, j] = X[i].dot(X[j])  #linear kernel
                else:
                    K[j, i] = K[i, j]
        return K
    
    def compute_u(self,X,y):
        u = np.zeros((X.shape[0],))
        for j in range(X.shape[0]):
            u[j]=np.sum(y*self.alpha*self.K[:,j])+self.b
        return u
    
    def checkKKT(self, u, y, i): #检查是否满足KKT条件
        if (self.alpha[i]==0) and (y[i]*u[i]<1):
            return False
        elif (self.alpha[i]==self.C) and (y[i]*u[i]>1):
            return False
        elif (self.alpha[i]>0 and self.alpha[i]<self.C) and (y[i]*u[i]!=1):
            return False
        return True
            
    


In [40]:
svc=SVC(C=1)
svc.fit(X,y)
print('alpha:',svc.alpha)
print('b:',svc.b)
pred_y=svc.predict(np.array([[1,0],[-0.2,-0.1],[0,1]]))
pred_y=np.sign(pred_y)

alpha: [-0.18949988  0.0028913   0.06700074 -0.05677673  0.50126607  0.67553841
  1.02954025  0.29152818 -0.99058403 -0.00275203]
b: 3.7169260092378282


In [35]:
pred_y

array([1., 1., 1.])